In [1]:
import pandas as pd
import numpy as np
import pymysql
import pymongo
import re
import jieba
import logging
jieba.setLogLevel(logging.INFO)

In [2]:
mongo_client = pymongo.MongoClient("localhost", 27017)

In [3]:
db = mongo_client.db_huangsy

In [4]:
sz_rent_group = db.douban_sz_rent_group_v1

In [5]:
doc_cur = sz_rent_group.find({})

In [6]:
dict_rent = {}
title_list = list()
author_list = list()
response_count_list = list()
page_num_list = list()
response_time_list = list()
url_list = list()
dt_list = list()
content_list = list()
for item in doc_cur:
    title_list.append(item["title"])
    author_list.append(item["author"])
    response_count_list.append(item["response_count"])
    response_time_list.append(item["response_time"])
    page_num_list.append(item["page_num"])
    url_list.append(item["url"])
    dt_list.append(item["dt"])
    content_list.append(item["content"])
dict_rent["title"] = title_list
dict_rent["author"] = author_list
dict_rent["response_count"] =response_count_list
dict_rent["response_time"] =response_time_list
dict_rent["page_num"] =page_num_list
dict_rent["url"] =url_list
dict_rent["dt"] =dt_list
dict_rent["content"] =content_list

In [7]:
df_rent = pd.DataFrame(dict_rent)

In [8]:
def parse_topic_id(topic_url):
    s = re.search(r'/topic/(\d+)/', topic_url).group(1)
    return s

In [9]:
def confirm_date(time_str):
    s = re.match(r'\d{4}\-\d{2}-\d{2} \d{2}:\d{2}', time_str)
    if s is None:
        return False
    else:
        return True


def parse_response_time(time_str):
    s1 = re.match(r'\d{4}\-\d{2}-\d{2} \d{2}:\d{2}', time_str)
    s2 = re.match(r'\d{4}\-\d{2}-\d{2}\s*', time_str)
    if s2  is not None:
        time_extract = re.match(r'\d{4}\-\d{2}-\d{2}', time_str).group(0)
        return time_extract + " 00:00"
    elif s1 is None:
        return '2019-' + time_str
    else:
        return time_str

    
def get_year(date_str):
    return date_str[0:4]

In [10]:
df_rent["response_time"] = df_rent["response_time"].apply(parse_response_time)

In [11]:
df_rent["post_year"] = df_rent["response_time"].apply(get_year)

In [12]:
df_rent['topic_id'] = df_rent['url'].apply(parse_topic_id)

In [13]:
df_rent = df_rent.drop_duplicates('topic_id')

In [14]:
 def rent_for_girl(series_text):
    title = series_text["title"]
    content = series_text["content"]
    content = title + '。' + content
    s1 = re.search(r'(?<!不)[只限仅][\u4E00-\u9FA5\s,，、]*(?:女|小姐姐|妹子)|(?<!不)[招寻找][\u4E00-\u9FA5\s,，、]*(?:女|小姐姐|妹子)|不[\u4E00-\u9FA5](?:男生|汉子|小哥哥|男的)合租', content)
    s2 = re.search(r'男女不限|不限男女|男女皆可|男女都行|男女都可', content) 
    if s2 is not None:
        return 0
    else:
        if s1 is not None:
            return 1
        else:
            return 0
        
        
def rent_for_man(series_text):
    title = series_text["title"]
    content = series_text["content"]
    content = title + '。' + content
    s1 = re.search(r'(?<!不)[只限仅][\u4E00-\u9FA5\s,，、]*(?:男|小哥哥|汉子)|(?<!不)[招寻找][\u4E00-\u9FA5\s,，、]*(?:男|小哥哥|汉子)|不[\u4E00-\u9FA5](?:女生|妹子|小姐姐|女的)合租', content)
    s2 = re.search(r'男女不限|不限男女|男女皆可|男女都行|男女都可', content) 
    if s2 is not None:
        return 0
    else:
        if s1 is not None:
            return 1
        else:
            return 0

In [15]:
def get_rent_type(series_text):
    title = series_text["title"]
    content = series_text["content"]
    rent_type_rule = {
        "转租": r"转[^\.。\??\!！\)）]*[出租]|转[^\.。\??\!！\)）]*[间房卧室]|急转|全[包齐含]",
        "招租": r"[招找寻求][^\.。\??\!！\)）]*(?:舍友|室友|小哥哥|小姐姐|帅哥|美女|女生|男生|妹子)|[招出合分]租|房东(?:发布|直租)|全[包齐含]", # 排除掉转租再来判断招租
        "求租": r"""求[短]*租|[找寻求][^\.。\??\!！\)）]*(?:单间|单房|房子|房|次卧|主卧|卧室|附近|床位|区|公寓)|有没有.*[租房]|[租房间厅卫卧室][\u4E00-\u9FA5\s]*[\?？]|预算|(\d+)[元]*以内""" #排除掉招转再判断是否求租贴
    }
    rent_type_rule2 = {
        "转租": r"转[^\.。\??\!！\)）]*[出租]|转[\u4E00-\u9FA5\s]*[间房卧室]|急转|全[包齐含]",
        "招租": r"[招找求寻][^\.。\??\!！\)）]*室友|[招出合分]租|房东(?:发布|直租)|全[包齐含]", # 排除掉转租再来判断招租
        "求租": r"求[短]*租|[找寻求][^\.。\??\!！\)）]*(?:单间|单房|房子|房|次卧|主卧|卧室|附近|床位|区|公寓)" #排除掉招转再判断是否求租贴
    }
    match_form = {
        "转租": 0,
        "求租": 0,
        "招租": 0
    }
    match_key2 = list()
    # 优先标题
    for rent_type in rent_type_rule.keys():
        rule = rent_type_rule[rent_type]
        flist = re.findall(rule, title)
        match_form[rent_type] = len(flist)
    rent_type = max(match_form, key=match_form.get)
    match_val = match_form[rent_type]
    for key in match_form.keys():
        if match_form[key] == match_val:
            match_key2.append(key)
    # 如果标题匹配到只有一种类型，以该类型为准
    if len(match_key2) == 1:
        return rent_type
    
    # 标题匹配不到类型，从内容匹配
    for rent_type in match_key2:
        rule = rent_type_rule2[rent_type]
        flist = re.findall(rule, content)
        match_form[rent_type] = len(flist)
    rent_type = max(match_form, key=match_form.get)
    match_val = match_form[rent_type]
    if match_val > 0:
        return rent_type
    else:
        # 标题内容都匹配不到，标题里出现以下关键词的更可能是招租帖子
        s = re.search(r"[租单间房卧室厅卫床地铁公交线站舍友区城村]", title)
        if s is not None:
            return "招租"
        else:
            return ""

In [16]:
def get_rent_style(series_text):
    title = series_text["title"]
    content = series_text["content"]
    rent_style_rule = {
        "分租": r"单[房间]|单租[\u4E00-\u9FA5\s,，、]*(?:房间|房子)|合租|[次主]卧|卧室|[1-7一二三四五六七两][室房](?:[1-3一二三两][厅卫])*[\u4E00-\u9FA5\s,，、]+[一1][\u4E00-\u9FA5\s,，、]+房|[室舍]友",
        "整租": r"[1-7一二三四五六七两]居*[室房](?:[1-3一二三两][厅卫])*|整租",
    }
    rent_style_except={
        "分租": r"^$",
        "整租": r"[1-7一二三四五六七两]居*[室房](?:[1-3一二三两][厅卫])*[\u4E00-\u9FA5\s,，、]*?(?:次卧|主卧|房)",
     
    }
    match_form = { "分租": 0,
        "整租":0}
    s1 = re.search(rent_style_rule["分租"], title)  
    s2 = re.search(rent_style_rule["整租"], title)
    if s1 is not None:
        return "分租"
    elif s2 is not None:
        return "整租"
    content = title + "。" + content

    for style in rent_style_rule:
        rule = rent_style_rule[style]
        except_rule = rent_style_except[style]
        flist = re.findall(rule, content)
        elist = re.findall(except_rule, content)
        if (len(flist) - len(elist)) !=0:
    
            match_form[style] = len(flist) - len(elist)

    rent_style = max(match_form, key=match_form.get)
    match_val = match_form[rent_style]
    if match_val == 0:
        return ""
    else:
        return rent_style

In [17]:
class MySQLConn(object):
    """Class supply connection to mysql database"""
    def __init__(self, **info):
        self.host = info.get("host", "localhost")
        self.user = info.get("user", "root")
        self.port = info.get("port", 3306)
        self.password = info.get("password", "*******")
        self.db = info.get("db", "bi_huangsy")
        self.table = info.get("table", "spu_tp_shop_map_cols")
        self.table_add = info.get("table_add", None)
        self.charset = info.get("charset", "utf8")
        try:
            self.conn = pymysql.connect(host=self.host, port=self.port,
                                        user=self.user, password=self.password, db=self.db, charset=self.charset)
            self.cur = self.conn.cursor()
        except pymysql.err.OperationalError as e:
            print('Error is ' + str(e))
            sys.exit()

    def read_mysql(self, col=None, where_append=None, chunk_size=500000):
        if col is None:
            col = "*"
        if where_append is None:
            where_append = ''
        try:
            sql = 'select %s from %s %s' % (col, self.table, where_append)
            dfs = pd.read_sql(sql, con=self.conn, chunksize=chunk_size)
        except pymysql.err.ProgrammingError as e:
            print('Error is ' + str(e))
            sys.exit()
        dfs = list(dfs)
        if len(list(dfs)) == 0:
            return None
        else:
            return pd.concat(dfs)

    def to_sql(self, df, *args):
        path = 'C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/tmp.csv' 
        df.to_csv(path, sep=',', index=False)
        sql = r"load data infile '%s' into table %s fields terminated by ',' lines terminated by '\r\n' ignore 1 lines;" \
            % (path, self.table)
        self.cur.execute(sql)
        self.conn.commit()

    def close(self):
        self.conn.close()
        # output.getvalue()
        # # jump to start of stream
        # output.seek(0)
        # engine_connect = "mysql+pymysql://" + self.user + ":" + self.password + "@" + self.host + "/" + self.db
        # engine = create_engine(engine_connect)
        # connection = engine.raw_connection()  # engine 是 from sqlalchemy import create_engine
        # cursor = connection.cursor()
        # # null value become ''
        # cursor.copy_from(output, self.table, null='')
        # connection.commit()

In [18]:
info = {"table": "db_sz_metro_station"}
mysql  = MySQLConn(**info)
df_metro = mysql.read_mysql()

In [19]:
metro_line_agg = df_metro[['metro_station','line']].groupby('metro_station').apply(lambda x: '|'.join([str(y) for y in x.line.tolist()])).reset_index()
metro_line_agg.columns=["metro_station", "line"]
metro_line_agg.head()

,metro_station,line
0,上塘站,4
1,上梅林站,4|9
2,上水径站,5
3,上沙站,7
4,下梅林站,9


In [20]:
METRO_LIST = list(set(df_metro.metro_station.tolist()))

In [21]:
METRO_STR = '|'.join([metro.replace('站', '') for metro in METRO_LIST])

In [22]:
RULE_METRO_STR1 = r'((?:%s))[双两]*(?:地铁|站|[a-zA-Z出]+口)'% METRO_STR
RULE_METRO_STR2 = r'号线\s*((?:%s))' % METRO_STR
RULE_METRO_STR3 = r'地铁[\u4E00-\u9FA5]*((?:%s))' % METRO_STR
# RULE_METRO_STR = r'(?:%s)'% RULE_METRO_STR

In [23]:
from copy import copy
METRO_LIST_NST = copy(METRO_LIST)
district_list = ["南山", "福田", "罗湖","宝安","龙华","龙岗","盐田","光明","坪山","大鹏"]
for district in district_list:
    if district + "站" in METRO_LIST_NST:
        METRO_LIST_NST.remove(district + "站")
METRO_RULE_NST =  '|'.join([metro.replace('站', '') for metro in METRO_LIST_NST])

In [24]:
def get_district(series_text):
    """获取地区"""
    title = series_text["title"]
    content = series_text["content"]
    district_rule = "南山|福田|罗湖|宝安|龙华|龙岗|盐田|光明|坪山|大鹏"
    d1 = re.findall(district_rule, title)
    d2 = re.findall(district_rule, content)
    d1.extend(d2)
    district_count = {}
    if len(d1) is 0:
        return ""
    for district in d1:
        district_count[district] = district_count.get(district, 0) + 1
    key_district = max(district_count, key=district_count.get)
    return key_district

def get_metro_station_st(series_text):
    global RULE_METRO_STR1, RULE_METRO_STR2, RULE_METRO_STR3
    title = series_text["title"]
    content = series_text["content"]
    content = title + '。' + content  
    rm_obj1 = re.findall(RULE_METRO_STR1, content)
    rm_obj2 = re.findall(RULE_METRO_STR2, content)
    rm_obj3 = re.findall(RULE_METRO_STR3, content)
    rm_obj1.extend(rm_obj2)
    rm_obj1.extend(rm_obj3)
    rm_obj = rm_obj1
    if len(rm_obj) == 0:
        return ""
    else:
        station_pack = {}
        for st in rm_obj:
            station_pack[st] = station_pack.get(st, 0) + 1
        most_freq_station = max(station_pack,key=station_pack.get)
        return most_freq_station + "站"
#     d = re.findall(RULE_METRO_STR, content)
#     if len(d) is 0:
#         return ""
#     return "|".join([m + "站" for  m in list(set(d))])
#     rm_obj1 = re.search(RULE_METRO_STR1, content)
#     rm_obj2 = re.search(RULE_METRO_STR2, content)
#     rm_obj3 = re.search(RULE_METRO_STR3, content)
#     if rm_obj1:
#         return rm_obj1.group(1)+"站"
#     elif rm_obj2:
#         return rm_obj2.group(1)+"站"
#     elif rm_obj3:
#         return rm_obj3.group(1)+"站"
#     else:
#         return ""

def get_metro_station_nst(series_text):
    global METRO_RULE_NST
    title = series_text["title"]
    content = series_text["content"]
    content = title + '。' + content 
    rm_obj = re.findall(METRO_RULE_NST, content)
    if len(rm_obj) == 0:
        return ""
    else:
        station_pack = {}
        for st in rm_obj:
            station_pack[st] = station_pack.get(st, 0) + 1
        most_freq_station = max(station_pack,key=station_pack.get)
        return most_freq_station + "站"
        
        
def get_metro_station(series_text): 
    sta = get_metro_station_st(series_text)
    if sta != "":
        return sta
    else:
        sta = get_metro_station_nst(series_text)
        return sta
     

In [25]:
title = "求深圳东门附近十平米左右的小仓库，水电网全包，价格1000以内可接受"
content="""小型办公室，主要做为电商发货，放一个货架一台电脑即可。"""
test = {"title": title, "content": content}
get_rent_type(test)

'求租'

In [26]:
rent_type_rule = {
    "转租": r"转[^\.。\??\!！\)）]*[出租]|转[^\.。\??\!！\)）]*[间房卧室]|急转",
    "招租": r"[招找寻求][^\.。\??\!！\)）]*(?:舍友|室友|小哥哥|小姐姐|帅哥|美女|女生|男生|妹子)|[招出合分直]租|房东(?:发布|直租)", # 排除掉转租再来判断招租
    "求租": r"""求[短]*租|[找寻求][^\.。\??\!！\)）]*(?:单间|单房|房子|房|次卧|主卧|卧室|附近|床位|区|公寓)|有没有.*[租房]|[租房间厅卫卧室][\u4E00-\u9FA5\s]*[\?？]|预算""" #排除掉招转再判断是否求租贴
}
rent_type_rule2 = {
    "转租": r"转[^\.。\??\!！\)）]*[出租]|转[\u4E00-\u9FA5\s]*[间房卧室]|急转",
    "招租": r"[招找求寻][^\.。\??\!！\)）]*室友|[招出合分直]租|房东(?:发布|直租)", # 排除掉转租再来判断招租
    "求租": r"求[短]*租|[找寻求][^\.。\??\!！\)）]*(?:单间|单房|房子|房|次卧|主卧|卧室|附近|床位|区|公寓)" #排除掉招转再判断是否求租贴
}

In [27]:
re.findall(rent_type_rule["求租"], title)

['求深圳东门附近']

In [28]:
title = "【个人转租】近南山地铁站三居室次卧短租3个月2500免押金"
content="""近11号线南山站和1号线桃园站，小区房三居室的次卧，房间面积10平方，
7.1以后可入住，租约到10.31，房子是跟房东整租下来的，到期后继续住需自行跟房东续签。
房子目前已入住三个女生和一只猫，租金2500，免押金，限女生，不可带宠物"""
test = {"title": title, "content": content}
get_rent_style(test)

'分租'

In [29]:
rent_style_rule = {
    "分租": r"单[房间]|单租[\u4E00-\u9FA5\s,，、]*(?:房间|房子)|合租|[次主]卧|卧室|[1-7一二三四五六七两][室房](?:[1-3一二三两][厅卫])*[\u4E00-\u9FA5\s,，、]+[一1][\u4E00-\u9FA5\s,，、]+房|室友",
    "整租": r"[1-7一二三四五六七两]居*[室房](?:[1-3一二三两][厅卫])*|整租",
}

In [30]:
re.findall(r"[1-7一二三四五六七两]居*[室房](?:[1-3一二三两][厅卫])*", title+"。"+content)

['三居室', '三居室']

In [31]:
test2= {"title": "【合租/南山区.次卧】龙联花园，2000元，限女生", "content": "南山宝安合租的同学可以看过来"}
get_district(test2)

'南山'

In [32]:
test3 =  {"title": "# 招租啦 #", "content": "无中介费 可入住 仅限女生♚♚♚ 可两人女生，不考虑情侣 不考虑男生……男生就不用往下看了 适合闺蜜姐妹住，目前有三人在住，主卧两人 ，小次卧一人 可领包入住，三室两卫一厅，出租次卧 房租2300  水电费另外按人头平分，水电费物业费(夏天开空调120/月/人，冬天不开空调80元月/人) 1、坐标南山西丽留仙居，400米可达5/7号线西丽地铁站，小区楼下3分钟有公交站，6分钟可达西丽医院公交站，10分钟路程有西丽法庭站 2、吃的玩的喝的乐的小区门口一条街，电影院，ktv 天虹，人人乐;悦方广场步行10分钟可达3、主卧，次卧共住有3个女生，大家都非常友好相处，都有固定工作，分别是设计，程序猿工作4、小区房相对安全😺房子在六楼，无电梯😂，优势是水电相对便宜，整个套间96平，是三房两卫一厅，家具齐全，空调、冰箱、洗衣机、电视机等，有大阳台大厨房5、😜出租的大次卧😋有飘窗，有11平方，有床、空调、衣柜、拎包可入住。有两张书桌（这是前面的女生买的价格可谈）房租价格是2300元。另水电约100月/人 6、希望住进来的你是个妹子，有正当工作，无不良嗜好，好相处 爱干净，不养宠物，一起爱护我们的大集体环境～7、本人微信号:JH531483826加我的时候备注:豆瓣租房"}

In [33]:
get_metro_station(test3)

'西丽站'

In [34]:
df_rent["rent_type"] = df_rent[["title", "content"]].apply(get_rent_type, axis=1)
df_rent["rent_style"] = df_rent[["title", "content"]].apply(get_rent_style, axis=1)

In [35]:
df_rent.rent_type.value_counts()

招租    59429
转租    33180
求租    25832
       7268
Name: rent_type, dtype: int64

In [36]:
df_rent.rent_style.value_counts()

分租    74365
整租    28169
      23175
Name: rent_style, dtype: int64

In [37]:
df1 = df_rent.loc[df_rent.rent_type == "招租"].sample(100)
df2 = df_rent.loc[df_rent.rent_type == "求租"].sample(100)
df3 = df_rent.loc[df_rent.rent_type == "转租"].sample(100)
df_rent_type = pd.concat([df1, df2, df3], axis=0)

In [38]:
df_rent_type.to_excel(r"D:\pandas data\rent_type.xlsx")

In [39]:
def parse_meaningless_content(text):
    rule0 = r'[^0-9][1-9][0][1-9][^0-9]'
    #new_text = re.sub("\d{2,4}-*\d{2,4}[多]*[平方mM分钟米㎡²兆号]+|[a-zA-Z\s_][0-9a-zA-Z\-－\s_]{6,}|10086", "", text)
    rule1 = r"[^。！.?!？;；]+\d+[千万kw]元*[^。！.?!？;；]*(?:一平|/每*平|平方|平米)"
    rule2 = r"(?:一平|每平|平方|平米)[^。！.?!？;；]*\d+[千万kw]元*"
    rule3 =  r"(?:\d{2,4}[-—~])*\d{2,4}[余多]*[平方mM分钟米㎡²兆人年]+|[a-zA-Z_0-9][a-zA-Z_0-9\s]{5,}|10086|(?:倒贴|补贴|优惠|减免|直减|省[去掉]*|水电费*|物业费*)\d+|[a-zA-Z]\d+|\d+[a-zA-Z]"
    rule4 = r"(?:电话|联系人|联系方式|手机)[:：\s]+[0-9-—]+|[:：\s]+[a-zA-Z]+[0-9—_a-zA-Z-]+"
    rule = '|'.join([rule0, rule1, rule2, rule3,rule4])
    new_text = re.sub(rule, "", text)
    return new_text

In [40]:
df_rent['district'] = df_rent[["title", "content"]].apply(get_district, axis=1)

In [41]:
df_rent['metro_station'] = df_rent[["title", "content"]].apply(get_metro_station, axis=1)

In [42]:
district_metro = df_rent.loc[df_rent.district != ""][["district", "metro_station"]].groupby("metro_station").apply(lambda x: x.district.value_counts().index[0]).reset_index()
district_metro.columns=["metro_station", "district2"]
district_metro = district_metro.loc[district_metro.metro_station !=""]
district_metro.head()

,metro_station,district2
1,上塘站,龙华
2,上梅林站,福田
3,上水径站,龙岗
4,上沙站,福田
5,下梅林站,福田


In [43]:
print(df_rent.loc[df_rent.district == ""].shape[0])
df_rent = df_rent.merge(district_metro, how="left", on='metro_station')
df_rent.loc[df_rent.metro_station != "", "district"] = df_rent.loc[df_rent.metro_station != "", "district2"]
df_rent.drop(columns="district2", inplace=True)
print(df_rent.loc[df_rent.district == ""].shape[0])

60761
20884


In [44]:
df_rent.head()

,title,author,response_count,response_time,page_num,url,dt,content,post_year,topic_id,rent_type,rent_style,district,metro_station
0,急转租 固戍地铁口低价豪华公寓,豆友192477417,17,2019-06-15 19:51,0,https://www.douban.com/group/topic/143147944/,2019-06-15 20:07:19,因工作地点变动，转租现在的房间。明年4月份到期，可以直接跟房东签约。房间情况：今年3月份刚装...,2019,143147944,转租,分租,宝安,固戍站
1,【个人转租】租金1400，首月可补贴800，石厦地铁站，近福田CBD，随时可入住，有优惠，新...,寂静的风,11,2019-06-15 19:56,0,https://www.douban.com/group/topic/140787304/,2019-06-15 20:07:19,因为要和同事合租，只能转租，我还没住进去，随时可以入住。小区房，房间不大，公用厨房可以做饭，...,2019,140787304,转租,分租,福田,石厦站
2,坪洲地铁站附近径贝新村一房一厅个人直租,豆友197023060,31,2019-06-15 19:46,0,https://www.douban.com/group/topic/141831059/,2019-06-15 20:07:19,径贝新村十二巷八号（导航至：勤业商务大厦）一房一厅出租，家电家私齐全，配备网络，监控，离坪洲...,2019,141831059,招租,整租,宝安,坪洲站
3,宝安灵芝地铁旁精装公寓转租,约翰君,26,2019-06-15 19:55,0,https://www.douban.com/group/topic/141853020/,2019-06-15 20:07:19,交通：灵芝地铁站旁，楼下诺铂广场公交站，去科技园方便，半个小时到科技园环境：新装修才半年，家...,2019,141853020,转租,,宝安,灵芝站
4,(转租)宝安坪洲地铁口两房一厅,MDZZᝰ,4,2019-06-15 19:46,0,https://www.douban.com/group/topic/143002741/,2019-06-15 20:07:19,转租两房一厅，阳光好，大阳台，离地铁非常近，宝安坪洲B出口，出门左拐就是地铁。现因舍友换工作...,2019,143002741,转租,整租,宝安,坪洲站


In [45]:
df_rent.loc[df_rent.district == ""].head()

,title,author,response_count,response_time,page_num,url,dt,content,post_year,topic_id,rent_type,rent_style,district,metro_station
36,【求租】预算1500以内，女生合租,阿吉,None,2019-06-15 19:41,1,https://www.douban.com/group/topic/143320578/,2019-06-15 20:08:04,刚来深圳，在科技园上班，求一号线附近的房子，最好全是女生（最好而已），望找合租的女生找我一起～,2019,143320578,求租,分租,,
80,求租～～～最好是短租～快来找我吧～,🐈,27,2019-06-15 19:00,3,https://www.douban.com/group/topic/142789847/,2019-06-15 20:08:41,,2019,142789847,求租,,,
89,（个人转租）流塘新村栖息地公寓一房一厅2380元,黄清清🎀🌈,5,2019-06-15 19:20,2,https://www.douban.com/group/topic/143318953/,2019-06-15 20:08:10,流塘新村，靠近港隆城，端头位一房一厅转租，精装修，可拎包入住!房间里设施齐全，有床、空调、洗...,2019,143318953,转租,整租,,
158,求圣淘沙/汇一城合租单间,小油条,None,2019-06-15 17:56,7,https://www.douban.com/group/topic/143313424/,2019-06-15 20:10:12,本人寻圣淘沙或者合正汇一城的合租单间，价位在1500到2500可以接受，最好两室一厅或三室一...,2019,143313424,招租,分租,,
164,# 招租啦 #,星。,10,2019-06-15 17:59,7,https://www.douban.com/group/topic/142551953/,2019-06-15 20:10:12,房型: 一室一厅两卫，标准的复式房（非Loft）。一楼客厅，二楼卧房。上下两层都有卫生间。领...,2019,142551953,招租,,,


In [46]:
df_rent['girl_only'] = df_rent[["title", "content"]].apply(rent_for_girl, axis=1)
df_rent['man_only'] = df_rent[["title", "content"]].apply(rent_for_man, axis=1)

In [47]:
df1 =df_rent.loc[df_rent.girl_only==1].sample(100)
df2 = df_rent.loc[df_rent.girl_only==0].sample(100)
df_rent_xlsx = pd.concat([df1,df2], axis=0)
df_rent_xlsx.to_excel(r"D:\pandas data\girl_only.xlsx")                                  

In [48]:
print(df_rent.loc[df_rent.topic_id=="143308672"].content.tolist()[0])

【求租】7月入住，5号线大学城，西丽，兴东，洪浪北地铁站求租一房一厅，套房合租也可，两个女生，预算3000以内（包括水电管理费等）

目前的房子7月10号到期，所以想赶紧寻到合适的房源，可以提前搬入，两个女生一起，一定要离地铁口近，不要超过5分钟！！！两个人都要做地铁上班的，一房一厅或者套房租其中一间都可以，不考虑和男生合租套房，不考虑农民房，最好是民用水电费哈，求合适房源！！！
联系方式：LoisYang0307
    或微信：13415257199


In [49]:
df_rent.loc[df_rent.girl_only==1].head()

,title,author,response_count,response_time,page_num,url,dt,content,post_year,topic_id,rent_type,rent_style,district,metro_station,girl_only,man_only
15,【合租/南山区.次卧】龙联花园，2000元，限女生,大侠虾,15,2019-06-15 19:53,0,https://www.douban.com/group/topic/143136207/,2019-06-15 20:07:19,@深圳租房团 诚心合租，限女生合租，非诚勿扰！中介勿扰！【7号线珠光站地铁口】真诚的！➡️三...,2019,143136207,招租,分租,南山,珠光站,1,0
26,转租.直租.福田区益田村.单间.1500元 限女生,大恒恒,40,2019-06-15 19:24,2,https://www.douban.com/group/topic/138634440/,2019-06-15 20:08:10,小区情况：房子位于益田村78栋二单元，楼下步行几分钟有益田广场、社区图书馆、游泳馆。小区都有...,2019,138634440,转租,分租,福田,益田站,1,0
27,【求租！！！】7月入住，5号线大学城，西丽，兴东，洪浪北地铁站求租一房一厅，套房合租也可，两...,Winni,2,2019-06-15 19:34,2,https://www.douban.com/group/topic/143308672/,2019-06-15 20:08:10,【求租】7月入住，5号线大学城，西丽，兴东，洪浪北地铁站求租一房一厅，套房合租也可，两个女生...,2019,143308672,求租,分租,宝安,洪浪北站,1,0
29,【福田CBD 益田地铁口】益田村小区 精装纯木复式楼 主卧床位转租 1100 限女生,吃掉这首歌,5,2019-06-15 19:37,1,https://www.douban.com/group/topic/143269626/,2019-06-15 20:08:04,1.小区益田村小区，门口就是3号线益田地铁站，步行只需5分钟，距离市中心区3公里左右。小区门...,2019,143269626,转租,分租,福田,益田站,1,0
35,茶光地铁站附近，两房一厅，有个主卧出租，只限女孩纸，有养猫！,豆友197672298,28,2019-06-15 19:27,2,https://www.douban.com/group/topic/143045632/,2019-06-15 20:08:10,(⁎⁍̴̛ᴗ⁍̴̛⁎)有想要合租的妹纸留言～,2019,143045632,招租,分租,南山,茶光站,1,0


In [50]:
def get_daily_rent(series_text):
    title = series_text["title"]
    content = series_text["content"]
    content = title + '。' + content
    content = parse_meaningless_content(content)
    rule1 = r"租[\u4E00-\u9FA5,，、]*([1-9]\d{2}|[4-9]\d{1})元/[每]*[天日]"
    rule2 = r"[日天][\u4E00-\u9FA5]*租[\u4E00-\u9FA5]*([1-9]\d{2}|[4-9]\d{1})元"
    s = re.search(rule1, content)
    if s is None:
        s = re.search(rule2, content)
        return "" if s is None else s.group(1)
    else:
        return s.group(1)
    
def parse_daily_rent(daily_rent):
    if daily_rent != "":
        return int(daily_rent)
    else:
        return np.nan
    
    

def get_monthly_rent(series_text):
    title = series_text["title"]
    content = series_text["content"]
#     content = title + '。' + content
#     content = content.replace(" ", "")
    ntitle = parse_meaningless_content(title)
    ncontent = parse_meaningless_content(content)
    rent_m = get_monthly_rent_main(ntitle)
    if rent_m == "":
        rent_a = get_monthly_rent_a(ntitle)
        if rent_a == "":
            rent_m = get_monthly_rent_main(ncontent)
            return rent_m if rent_m != "" else get_monthly_rent_a(ncontent)
        else:
            return rent_a
    else:
        return rent_m

    
def get_monthly_rent_main(text):
   
    rule1 = r"""(?:卧|厅|卫|室|房|间|包|月|租|小区|园|厦|公交|村|地铁|预算|附近|公寓|装修|以内|左右)[\u4E00-\u9FA5，,、()（）【】\s¥:：]*([5-9]\d{2}|[1-9]\d{3,4}|\d{1,2}\.\d+[千k]|\d{1,2}[千k]|\d{1,2}\.\d+[万w]|\d{1,2}[万w])(?:[余多]*[^平方mM米年0-9]|$)"""
    rule2 = r"""([5-9]\d{2}|[1-9]\d{3,4}|\d{1,2}\.\d+[千k]|\d{1,2}[千k]|\d{1,2}\.\d+[万w]|\d{1,2}[万w])[多余]*[^平方mM米人年0-9][\u4E00-\u9FA5，,、()（）【】\s¥:：]*(?:卧|厅|卫|室|房|间|包|月|租|小区|公交|园|厦|村|地铁|预算|附近公寓|装修|以内|左右)"""
    rule3 = r"([5-9]\d{2}|[1-9]\d{3,4}|\d{1,2}\.\d+[千k]|\d{1,2}[千k]|\d{1,2}\.\d+[万w]|\d{1,2}[万w])元*/每*月"
    rule4 = r"([5-9]\d{2}|[1-9]\d{3,4}|\d{1,2}\.\d+[千k]|\d{1,2}[千k]|\d{1,2}\.\d+[万w]|\d{1,2}[万w])元"
    #rule = "|".join([rule1, rule2, rule3, rule4])
    f1 = re.findall(rule1, text)
    f2 = re.findall(rule2, text)
    f3 = re.findall(rule3, text)
    f4 = re.findall(rule4, text)
    f1.extend(f2)
    f1.extend(f3)
    f1.extend(f4)
    # rule1,rule2，rule3,rule4有优先规则
    if len(f1) != 0:
        rent_pack = {}
        for rent in f1:
            rent_pack[rent] = rent_pack.get(rent, 0) + 1
        most_freq_rent = max(rent_pack, key = rent_pack.get)
        return most_freq_rent  #匹配出现次数最多的房租
    else:
        return ""  
    

def get_monthly_rent_a(text):
    """匹配如3000-4000元这种"""
    rule = r"[\u4E00-\u9FA5，,、()（）【】\s¥:：]([5-9]\d{2}|[1-9]\d{3})-([5-9]\d{2}|[1-9]\d{3})(?:[^\-0-9]|$)"
    s = re.search(rule, text)
    if s is not None:
        a, b = s.groups()[0], s.groups()[1]
        return str(int((int(a)+int(b))/2))
    else:
        return ""

def get_monthly_rent_h(text):
    rule1 = r"租[\u4E00-\u9FA5，,、()（）【】\s¥:：]*([5-9]\d{2})元*/月"
    rule2 = r"月[\u4E00-\u9FA5，,、()（）【】\s¥:：]*租[\u4E00-\u9FA5]*([5-9]\d{2})元"
    f1 = re.findall(rule1, text)
    f2 = re.findall(rule2, text)
    f1.extend(f2)
    if len(f1) != 0:
        rent_pack = {}
        for rent in f1:
            rent_pack[rent] = rent_pack.get(rent, 0) + 1
        most_freq_rent = max(rent_pack, key = rent_pack.get)
        return most_freq_rent
    else:
        return ""

def get_monthly_rent_w(text):
    rule3 = r"[^0-9a-zA-Z]([1-9]\d{4}|\d{1}\.\d+[万w]|\d{1}[万w])元*/每*月"
    rule1 = r"(?:卧|厅|卫|室|房|间|包|月|租|小区|园|厦|公交|村|地铁|预算|附近|公寓|装修|以内|左右)[\u4E00-\u9FA5，,、()（）【】\s¥:：]*([1-9]\d{4}|\d{1}\.\d+[万w]|\d{1}[万w])(?:[余多]*[^平方mM人米人]|$)"
    rule2 = r"([1-9]\d{4}|\d{1}\.\d+[万w]|\d{1}[万w])[多余]*[^^平方mM米人][\u4E00-\u9FA5，,、()（）【】\s¥:：]*(?:卧|厅|卫|室|房|间|包|月|租|小区|公交|园|厦|村|地铁|预算|附近公寓|装修|以内|左右)"
    rule4 = r"[^0-9a-zA-Z]([1-9]\d{4}|\d{1}\.\d+[万w]|\d{1}[万w])元"
    #rule = "|".join([rule1, rule2, rule3, rule4])
    f1 = re.findall(rule1, text)
    f2 = re.findall(rule2, text)
    f3 = re.findall(rule3, text)
    f4 = re.findall(rule4, text)
    f1.extend(f2)
    f1.extend(f3)
    f1.extend(f4)
    if len(f1) != 0:
        rent_pack = {}
        for rent in f1:
            rent_pack[rent] = rent_pack.get(rent, 0) + 1
        most_freq_rent = max(rent_pack, key = rent_pack.get)
        return most_freq_rent
    else:
        return ""

def get_monthly_rent_k(text):
    rule3 = r"[^0-9a-zA-Z]([1-9]\d{3}|\d{1}\.\d+[千k]|\d{1}[千k])元*/每*月"
    rule1 = r"(?:卧|厅|卫|室|房|间|包|月|租|小区|园|厦|公交|村|地铁|预算|附近|公寓|装修|以内|左右)[\u4E00-\u9FA5，,、()（）【】\s¥:：]*([1-9]\d{3}|\d{1}\.\d+[千k]|\d{1}[千k])(?:[余多]*[^平方mM米]|$)"
    rule2 = r"([1-9]\d{3}|\d{1}\.\d+[千k]|\d{1}[千k])[多余]*[^平方mM米人][\u4E00-\u9FA5，,、()（）【】\s¥:：]*(?:卧|厅|卫|室|房|间|包|月|租|小区|公交|园|厦|村|地铁|预算|附近公寓|装修|以内|左右)"
    rule4 = r"[^0-9a-zA-Z]([1-9]\d{3}|\d{1}\.\d+[千k]|\d{1}[千k])元"
    #rule = "|".join([rule1, rule2, rule3, rule4])
    f1 = re.findall(rule1, text)
    f2 = re.findall(rule2, text)
    f3 = re.findall(rule3, text)
    f4 = re.findall(rule4, text)
    f1.extend(f2)
    f1.extend(f3)
    f1.extend(f4)
    if len(f1) != 0:
        rent_pack = {}
        for rent in f1:
            rent_pack[rent] = rent_pack.get(rent, 0) + 1
        most_freq_rent = max(rent_pack, key = rent_pack.get)
        return most_freq_rent
    else:
        return ""  


In [52]:
df_rent["daily_rent"] = df_rent[["title", "content"]].apply(get_daily_rent, axis=1)

In [53]:
df_rent["daily_rent"] = df_rent["daily_rent"].apply(parse_daily_rent)

In [54]:
df_rent.loc[(df_rent.daily_rent != None)].head()

,title,author,response_count,response_time,page_num,url,dt,content,post_year,topic_id,rent_type,rent_style,district,metro_station,girl_only,man_only,daily_rent
0,急转租 固戍地铁口低价豪华公寓,豆友192477417,17,2019-06-15 19:51,0,https://www.douban.com/group/topic/143147944/,2019-06-15 20:07:19,因工作地点变动，转租现在的房间。明年4月份到期，可以直接跟房东签约。房间情况：今年3月份刚装...,2019,143147944,转租,分租,宝安,固戍站,0,0,NaN
1,【个人转租】租金1400，首月可补贴800，石厦地铁站，近福田CBD，随时可入住，有优惠，新...,寂静的风,11,2019-06-15 19:56,0,https://www.douban.com/group/topic/140787304/,2019-06-15 20:07:19,因为要和同事合租，只能转租，我还没住进去，随时可以入住。小区房，房间不大，公用厨房可以做饭，...,2019,140787304,转租,分租,福田,石厦站,0,0,NaN
2,坪洲地铁站附近径贝新村一房一厅个人直租,豆友197023060,31,2019-06-15 19:46,0,https://www.douban.com/group/topic/141831059/,2019-06-15 20:07:19,径贝新村十二巷八号（导航至：勤业商务大厦）一房一厅出租，家电家私齐全，配备网络，监控，离坪洲...,2019,141831059,招租,整租,宝安,坪洲站,0,0,NaN
3,宝安灵芝地铁旁精装公寓转租,约翰君,26,2019-06-15 19:55,0,https://www.douban.com/group/topic/141853020/,2019-06-15 20:07:19,交通：灵芝地铁站旁，楼下诺铂广场公交站，去科技园方便，半个小时到科技园环境：新装修才半年，家...,2019,141853020,转租,,宝安,灵芝站,0,0,NaN
4,(转租)宝安坪洲地铁口两房一厅,MDZZᝰ,4,2019-06-15 19:46,0,https://www.douban.com/group/topic/143002741/,2019-06-15 20:07:19,转租两房一厅，阳光好，大阳台，离地铁非常近，宝安坪洲B出口，出门左拐就是地铁。现因舍友换工作...,2019,143002741,转租,整租,宝安,坪洲站,0,0,NaN


In [55]:
print(df_rent.loc[df_rent.topic_id=="131707509"].content.tolist()[0])

这是一条给自己招室友的帖子，男女不限，可长期租房，因3号房间室友工作原因就要离开我们602的小家庭了，怀着不舍的心情来此寻觅新室友，帮她转租这个房间，另外有公用卫生间、厨房和大客厅大阳台，房子是正常的三室两厅一卫的户型，楼下是3号线，盒马生鲜，商场，电影院。如果你有需要可以联系我，微信：zxq15279191129 电话：13927470950。住602保证让你找到家的感觉，因为我做饭太香。


In [56]:
test = {"title":"南山-西丽 宝能华府两居室整租，可短租\n押一付一      3000元/月", "content": "这是一条给自己招室友的帖子，男女不限，可长期租房，因3号房间室友工作原因就要离开我们602的小家庭了，怀着不舍的心情来此寻觅新室友，帮她转租这个房间，另外有公用卫生间、厨房和大客厅大阳台，房子是正常的三室两厅一卫的户型，楼下是3号线，盒马生鲜，商场，电影院。如果你有需要可以联系我，微信：zxq15279191129 电话：13927470950。住602保证让你找到家的感觉，因为我做饭太香"}
parse_meaningless_content(test["title"])


'南山-西丽 宝能华府两居室整租，可短租\n押一付一      3000元/月'

In [57]:
df_rent["monthly_rent"] = df_rent[['title', 'content']].apply(get_monthly_rent, axis=1)

In [58]:
def parse_monthly_rent(rent_str):
    if rent_str == "":
        return np.nan
    else:
        if 'k' in rent_str or '千' in rent_str:
            new_str = re.sub('k|千', '', rent_str)
            rent = float(new_str)
            return int(rent*1000)
        elif 'w' in rent_str or '万' in rent_str:
            new_str = re.sub('w|万', '', rent_str)
            rent = float(new_str)
            return int(rent*10000)
        else:
            s= re.match('(\d{4})[1-9]', rent_str)
            if s is None:
                return int(rent_str)
            else:
                rent = int(s.group(1))
                return rent
    

In [59]:
df_rent["monthly_rent"] = df_rent["monthly_rent"].apply(parse_monthly_rent)

In [60]:
df_rent.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 125709 entries, 0 to 125708
Data columns (total 18 columns):
title             125709 non-null object
author            125709 non-null object
response_count    99478 non-null object
response_time     125709 non-null object
page_num          125709 non-null int64
url               125709 non-null object
dt                125709 non-null object
content           125709 non-null object
post_year         125709 non-null object
topic_id          125709 non-null object
rent_type         125709 non-null object
rent_style        125709 non-null object
district          125709 non-null object
metro_station     125709 non-null object
girl_only         125709 non-null int64
man_only          125709 non-null int64
daily_rent        23 non-null float64
monthly_rent      80534 non-null float64
dtypes: float64(2), int64(3), object(13)
memory usage: 18.2+ MB


In [61]:
pd.DataFrame(df_rent.monthly_rent.value_counts()).head()

,monthly_rent
1500.0,6398
2000.0,4904
1000.0,3240
1600.0,3198
1800.0,3157


In [62]:
rent_sz_myear_split = df_rent.loc[df_rent.rent_style.isin(["分租","分租|公寓"])][["post_year", "metro_station","monthly_rent"]].groupby(["post_year", "metro_station"]).mean().reset_index()
rent_sz_myear_split.loc[pd.notnull(rent_sz_myear_split.monthly_rent)].tail()

,post_year,metro_station,monthly_rent
1345,2019,黄贝岭站,1832.225806
1346,2019,龙井站,1952.850000
1347,2019,龙华站,1392.565657
1348,2019,龙城广场站,1357.142857
1349,2019,龙胜站,1567.016949


In [63]:
rent_sz_dyear_split = df_rent.loc[df_rent.rent_style.isin(["分租","分租|公寓"])][["post_year", "district","monthly_rent"]].groupby(["post_year", "district"]).mean().reset_index()
rent_sz_dyear_split.loc[rent_sz_dyear_split.district == "南山"].tail(10)

,post_year,district,monthly_rent
12,2010,南山,982.413793
20,2011,南山,1299.566845
28,2012,南山,1379.972222
37,2013,南山,1479.645631
46,2014,南山,1637.219512
56,2015,南山,1919.211124
66,2016,南山,2015.045455
76,2017,南山,1987.816374
87,2018,南山,2038.611447
98,2019,南山,2192.551432


In [64]:
rent_sz_year_split = df_rent.loc[df_rent.rent_style.isin(["分租","分租|公寓"])][["post_year","monthly_rent"]].groupby("post_year").mean().reset_index()
rent_sz_year_split.set_index('post_year', inplace=True)
#rent_sz_dyear_split.loc[rent_sz_dyear_split.district == "", "monthly_rent"] = rent_sz_dyear_split.loc[rent_sz_dyear_split.district == ""]['post_year'].apply(lambda x: rent_sz_year_split[x]["monthly_rent"])

In [65]:
#rent_sz_dyear_split.loc[rent_sz_dyear_split.district == "", "monthly_rent"] = rent_sz_dyear_split.loc[rent_sz_dyear_split.district == ""]['post_year'].apply(lambda x: rent_sz_year_split.loc[x,"monthly_rent"])

In [66]:
rent_sz_year_split = rent_sz_year_split.reset_index()
rent_sz_year_split.rename(columns={'monthly_rent': 'yearm_rent'}, inplace=True)
rent_sz_dyear_split = rent_sz_dyear_split.merge(rent_sz_year_split, how="left", on="post_year")
rent_sz_dyear_split.loc[pd.isnull(rent_sz_dyear_split.monthly_rent)|(rent_sz_dyear_split.district==""), "monthly_rent"] = rent_sz_dyear_split.loc[pd.isnull(rent_sz_dyear_split.monthly_rent)|(rent_sz_dyear_split.district=="")]["yearm_rent"]
rent_sz_dyear_split.drop(columns='yearm_rent', inplace=True)

In [67]:
rent_sz_dyear_split.head(10)

,post_year,district,monthly_rent
0,2008,,1087.777778
1,2008,南山,1296.666667
2,2008,宝安,1087.777778
3,2008,福田,925.000000
4,2008,罗湖,1000.000000
5,2009,,1048.723404
6,2009,南山,1317.857143
7,2009,宝安,1048.723404
8,2009,福田,945.000000
9,2009,罗湖,942.857143


In [68]:
rent_sz_myear_split.rename(columns={"monthly_rent": "ym_rent"}, inplace=True)
rent_sz_dyear_split.rename(columns={"monthly_rent": "ym_rent2"}, inplace=True)

In [69]:
df_rent = df_rent.merge(rent_sz_myear_split, how="left", on=['post_year', 'metro_station'])
df_rent = df_rent.merge(rent_sz_dyear_split, how="left", on=['post_year', 'district'])

In [70]:
def get_monthly_rent_modify(series_text):
    title = series_text["title"]
    content = series_text["content"]
    centre_month = series_text["ym_rent"]   # 匹配地铁站附近当年平均单房房租
    centre_month2 = series_text["ym_rent2"] # 所属行政区当年平均单房房租
    content = title + '。' + content
    ncontent = parse_meaningless_content(content)
    if np.isnan(centre_month): # 可能一些地铁站当年平均单房房租没有，则用区代替
        centre_month = centre_month2
    rent = get_monthly_rent_main_modify(ncontent, centre_month)
    return rent

def get_monthly_rent_main_modify(text, centre_month):
    """以围绕平均房租再清洗一次房租，返加离平均房租最近的房租"""
    rule1 = r"""(?:卧|厅|卫|室|房|间|包|月|租|小区|园|厦|公交|村|地铁|预算|附近|公寓|装修|以内|左右)[\u4E00-\u9FA5，,、()（）【】\s¥:：]*([5-9]\d{2}|[1-9]\d{3,4}|\d{1,2}\.\d+[千k]|\d{1,2}[千k]|\d{1,2}\.\d+[万w]|\d{1,2}[万w])(?:[余多]*[^平方mM米年0-9]|$)"""
    rule2 = r"""([5-9]\d{2}|[1-9]\d{3,4}|\d{1,2}\.\d+[千k]|\d{1,2}[千k]|\d{1,2}\.\d+[万w]|\d{1,2}[万w])[多余]*[^平方mM米人年0-9][\u4E00-\u9FA5，,、()（）【】\s¥:：]*(?:卧|厅|卫|室|房|间|包|月|租|小区|公交|园|厦|村|地铁|预算|附近公寓|装修|以内|左右)"""
    rule3 = r"([5-9]\d{2}|[1-9]\d{3,4}|\d{1,2}\.\d+[千k]|\d{1,2}[千k]|\d{1,2}\.\d+[万w]|\d{1,2}[万w])元*/每*月"
    rule4 = r"([5-9]\d{2}|[1-9]\d{3,4}|\d{1,2}\.\d+[千k]|\d{1,2}[千k]|\d{1,2}\.\d+[万w]|\d{1,2}[万w])元"
    #rule = "|".join([rule1, rule2, rule3, rule4])
    f1 = re.findall(rule1, text)
    f2 = re.findall(rule2, text)
    f3 = re.findall(rule3, text)
    f4 = re.findall(rule4, text)
    f1.extend(f2)
    f1.extend(f3)
    f1.extend(f4)
    rent_list = list()
    if len(f1) != 0:
        for rent in f1:
            rent_list.append(parse_monthly_rent(rent))
    rent_a = get_monthly_rent_a(text)
    if rent_a != "":
        rent_list.append(parse_monthly_rent(rent_a))
    if len(rent_list) == 0:
        return np.nan
    else:
        rent_arr = np.array(rent_list)
        return rent_arr[np.argmin(np.abs(rent_arr - centre_month))]
            

In [71]:
df_rent["monthly_rent_modify"] = np.nan

In [72]:
df_rent.loc[df_rent.rent_style.str.contains("分租"), "monthly_rent_modify"] = df_rent.loc[df_rent.rent_style.str.contains("分租")][["title", "content", "ym_rent", "ym_rent2"]].apply(get_monthly_rent_modify,axis=1)

In [73]:
def get_room_number(series_text):
    title = series_text["title"]
    content = series_text["content"]
    content = title + "。" + content
    rule = r"([1-7一二三四五六七两])居*[室房]"
    flist = re.findall(rule, content)
    if len(flist) !=0:
        nflist = list()
        for match_text in flist:
            nflist.append(parse_text_number(match_text))
        match_pack = {}
        for match_text in nflist:
            match_pack[match_text] = match_pack.get(match_text, 0) + 1
        match_number = max(match_pack, key=match_pack.get)
        return int(match_number)
    else:
        return 1
            
        
    
def parse_text_number(text):
    parse_rule = {"一": "1", "二": "2", "三": "3", "四": "4", "五": "5", "六": "6", "七": "7", "两": "2"}
    if text in parse_rule.keys():
        return parse_rule[text]
    else:
        return text

In [74]:
test={"title":"整租三房一厅6000元", "content":"附近的两房都要4000块了,这个三房可以说很实惠了"}
get_room_number(test)

3

In [75]:
df_rent["entire_room_num"] = 1
df_rent["entire_rent_split"] = np.nan
df_rent.loc[df_rent.rent_style.str.contains("整租"),"entire_room_num"] = df_rent.loc[df_rent.rent_style.str.contains("整租")][["title", "content"]].apply(get_room_number, axis=1)
df_rent.loc[df_rent.rent_style.str.contains("整租"),"entire_rent_split"] = (df_rent.loc[df_rent.rent_style.str.contains("整租")]["monthly_rent"]/df_rent.loc[df_rent.rent_style.str.contains("整租")]["entire_room_num"]).apply(lambda x: int(x) if x > 0 else x)

In [76]:
df_rent.loc[df_rent.entire_room_num != 1].head()

,title,author,response_count,response_time,page_num,url,dt,content,post_year,topic_id,...,metro_station,girl_only,man_only,daily_rent,monthly_rent,ym_rent,ym_rent2,monthly_rent_modify,entire_room_num,entire_rent_split
4,(转租)宝安坪洲地铁口两房一厅,MDZZᝰ,4,2019-06-15 19:46,0,https://www.douban.com/group/topic/143002741/,2019-06-15 20:07:19,转租两房一厅，阳光好，大阳台，离地铁非常近，宝安坪洲B出口，出门左拐就是地铁。现因舍友换工作...,2019,143002741,...,坪洲站,0,0,NaN,3700.0,1672.030303,1718.158282,NaN,2,1850.0
23,非自如非公寓【11号线碧海湾地铁站】 松茂御龙湾 3房两厅2卫,Patrice,27,2019-06-15 19:45,1,https://www.douban.com/group/topic/108893202/,2019-06-15 20:08:04,业主直租，无中介费，不私收水电费，\n\n看房直接去御龙湾物业管理处说拿钥匙。当然要提前联系...,2019,108893202,...,碧海湾站,0,0,NaN,NaN,2092.403509,1718.158282,NaN,3,NaN
38,福田益田村三室二厅一卫转租，等你来602吃火锅,朱朱朱晓庆,8,2019-06-15 19:27,2,https://www.douban.com/group/topic/131707509/,2019-06-15 20:08:10,这是一条给自己招室友的帖子，男女不限，可长期租房，因3号房间室友工作原因就要离开我们602的...,2019,131707509,...,益田站,0,0,NaN,NaN,2060.729167,2176.810216,NaN,3,NaN
56,近南山/桃园地铁站 复式公寓/两房一厅 无中介费。阳光科创中心,jokis,3,2019-06-15 19:15,3,https://www.douban.com/group/topic/143236201/,2019-06-15 20:08:41,近南山/桃园地铁站 复式公寓/两房一厅 无中介费。阳光科创中心六月底到期，房子在...,2019,143236201,...,桃园站,0,0,NaN,6300.0,2086.387821,2192.551432,NaN,2,3150.0
63,深大，滨海大厦旁【锦隆花园】4居室出租！！,弥留之际丶,1,2019-06-15 19:13,3,https://www.douban.com/group/topic/143103072/,2019-06-15 20:08:41,坐标【锦隆花园】，位于深圳大学旁，腾讯滨海大厦，软件园产业基地，周边上班的朋友们，看过来哦！...,2019,143103072,...,深大站,0,0,NaN,NaN,2226.064516,2192.551432,NaN,4,NaN


In [77]:
df_rent.loc[(df_rent.monthly_rent - df_rent.monthly_rent_modify > 1000) & pd.notnull(df_rent.monthly_rent_modify)].head(10)

,title,author,response_count,response_time,page_num,url,dt,content,post_year,topic_id,...,metro_station,girl_only,man_only,daily_rent,monthly_rent,ym_rent,ym_rent2,monthly_rent_modify,entire_room_num,entire_rent_split
113,【两房一厅招合租】,海棠,None,2019-06-15 18:39,5,https://www.douban.com/group/topic/143316230/,2019-06-15 20:09:16,房间情况：两房一厅，整租2500，两个房间大小一样，每个房间1250。有阳台、厨房、独立卫生...,2019,143316230,...,五和站,1,1,NaN,2500.0,1452.718631,1481.548148,1250.0,1,NaN
253,【找合租室友】坪洲地铁站C出口旁 轻铁西,哓巫,None,2019-06-15 16:09,12,https://www.douban.com/group/topic/143305622/,2019-06-15 20:11:14,先上图，农民房，轻铁西四巷七号，六楼无电梯，介意的无视掉！次卧招租，先说下费用房租总共300...,2019,143305622,...,坪洲站,0,0,NaN,3000.0,1672.030303,1718.158282,1400.0,1,NaN
384,八卦岭岭尚时代园招个舍友,Andy,1,2019-06-15 14:22,16,https://www.douban.com/group/topic/142821613/,2019-06-15 20:12:09,两房一，小区房，中高层，客厅出大阳台（京基100和平安大厦都看得到），家具全新自置，拎包可入...,2019,142821613,...,八卦岭站,0,0,NaN,6000.0,2110.588235,2176.810216,2500.0,1,NaN
406,出租、一号线『大新』阳光棕榈园，三房两卫，合租房,麻辣爽稻,11,2019-06-15 13:38,17,https://www.douban.com/group/topic/142992838/,2019-06-15 20:12:33,出租。一号线大新地铁站阳光棕榈园，三房两卫，有公共阳台，超级舒服房子呢是位于大新地铁站的阳光...,2019,142992838,...,大新站,0,0,NaN,3690.0,2382.168142,2192.551432,2560.0,1,NaN
631,【25平主卧出租 】4号线民乐地铁口 主次卧 限女生,catherine,9,2019-06-15 10:47,24,https://www.douban.com/group/topic/142307543/,2019-06-15 20:14:20,个人 直租房源，不用中介费！非公寓！1. 地铁口小区房子166平5房两厅两卫，现主次卧转租（...,2019,142307543,...,民乐站,1,0,NaN,2700.0,1541.690722,1668.835795,1650.0,1,NaN
658,【已出租，谢谢！--合租 罗湖两室一厅 电梯房 限女-地铁9号线泥岗站C出口】,被抢的阿毛,3,2019-06-15 09:15,26,https://www.douban.com/group/topic/142021259/,2019-06-15 20:16:19,房子6月底到期，室友要搬走，现在寻找合租室友。房子我们已经租了2年，房东人很好，平时很少过来...,2019,142021259,...,泥岗站,1,0,NaN,4000.0,1575.315789,1909.819831,2000.0,1,NaN
803,【个人转租】车公庙建业大厦2居室次卧2900元可住两人，即1450元／人，仅限妹子。和房东直...,豆友187883952,None,2019-06-14 22:30,32,https://www.douban.com/group/topic/143251261/,2019-06-15 20:17:36,【房子概况】房子是二房二厅一厨一卫一个大阳台一个小阳台，无隔断，主卧2张床（我睡其中一张），...,2019,143251261,...,农林站,1,0,NaN,2900.0,1922.181818,2176.810216,1450.0,1,NaN
920,龙华上塘地铁口品牌花园小区招租,单身奶爸🍼,37,2019-06-14 17:09,39,https://www.douban.com/group/topic/126667353/,2019-06-15 20:18:53,金地上塘道品牌花园，家私家电齐全拎包入住。小区安静舒适方便，走路去地铁口九方沃尔玛都是十分钟...,2019,126667353,...,上塘站,0,0,NaN,4800.0,1870.971014,1668.835795,1900.0,1,NaN
954,福田中心区小区两房 落地窗！夜景美！寻找合租整租小伙伴。福田南山罗湖 上下沙车公庙 新洲石厦...,Despicable me,116,2019-06-14 16:31,40,https://www.douban.com/group/topic/140517844/,2019-06-15 20:18:58,快来看房吧！！！小户型高档小区 品享受!品质享受！品质享受！目前是自己整租下来招室友 非二房...,2019,140517844,...,沙尾站,0,0,NaN,4400.0,2049.651429,2176.810216,2200.0,1,NaN
961,龙华上塘品牌花园小区两房一厅找合租,单身奶爸🍼,72,2019-06-14 17:08,39,https://www.douban.com/group/topic/132826266/,2019-06-15 20:18:53,金地上塘道品牌花园，家私家电齐全拎包入住。小区安静舒适方便，走路去地铁口九方沃尔玛都是十分钟...,2019,132826266,...,福田站,0,0,NaN,4600.0,1797.085106,2176.810216,1800.0,1,NaN


In [78]:
print(df_rent.loc[df_rent.topic_id=="141001652"].rent_style.tolist()[0])

分租


In [79]:
df_rent["entire_rent_split"] = df_rent["entire_rent_split"].astype(np.float)

In [80]:
df_rent["monthly_rent_modify"] = df_rent["monthly_rent_modify"].astype(np.float)

In [81]:
# df_rent.loc[df_rent.ym_rent == None, "ym_rent"] = np.nan
# df_rent.loc[df_rent.monthly_rent == None, "monthly_rent"] = np.nan
# df_rent.loc[df_rent.monthly_rent_modify == None, "monthly_rent_modify"] = np.nan
# df_rent.loc[df_rent.entire_rent_split == None, "entire_rent_split"] = np.nan
# df_rent.loc[df_rent.daily_rent == None, "daily_rent"] = np.nan

In [82]:
def check_verify_monthly_rent(series_text):
    centre_month = series_text["ym_rent"]  # 匹配地铁站附近当年平均单房房租
    centre_month2 = series_text["ym_rent2"]  # 所属行政区当年平均单房房租
    monthly_rent = series_text["monthly_rent"] # 首次清洗月租
    modify_rent = series_text["monthly_rent_modify"] # 基于区域平均值清洗的二次清洗的月租
    split_rent = series_text["entire_rent_split"] # 如果被判断整租，为首次清洗月租/房间数
    daily_rent = 30*series_text["daily_rent"] # 日租
    min_dist = 2000000 #选择一个尽量大的数
    if np.isnan(centre_month):
        centre_month = centre_month2
    rent_list = [monthly_rent, modify_rent, split_rent, daily_rent]
    nrent_list = list()
    # 去掉空值
    for x in rent_list:
        if not np.isnan(x):
            nrent_list.append(x)
    # 没有房租信息
    if len(nrent_list) == 0:
        return np.nan
    # 求最近，由于单房房租里几千的可能性比几百的可能性大，分别除于房底10对数，会优先选几千的。如单房平均房租1000，获取到了800的房租与
    # 与1200的房租，则1200的可能性更高。当然，这种方法对万房租的权重系数也更高点，但是万与平均房租的误差较大，所以影响不大
    rent_arr = np.array(nrent_list)
    wdist = np.log10(rent_arr+1).astype(np.int) + 1
    l = rent_arr[np.argmin(np.abs(rent_arr - centre_month)/wdist)]
    return l

In [83]:
test= {"ym_rent": 2270,"ym_rent2":2000, "monthly_rent": 650, "monthly_rent_modify": 2700, "entire_rent_split": np.nan, "daily_rent": np.nan}
check_verify_monthly_rent(test)

2700

In [84]:
df_rent["monthly_rent2"] = df_rent[["monthly_rent", "ym_rent","ym_rent2","monthly_rent_modify", "entire_rent_split", "daily_rent"]].apply(check_verify_monthly_rent,axis=1)

In [85]:
df_rent.head()

,title,author,response_count,response_time,page_num,url,dt,content,post_year,topic_id,...,girl_only,man_only,daily_rent,monthly_rent,ym_rent,ym_rent2,monthly_rent_modify,entire_room_num,entire_rent_split,monthly_rent2
0,急转租 固戍地铁口低价豪华公寓,豆友192477417,17,2019-06-15 19:51,0,https://www.douban.com/group/topic/143147944/,2019-06-15 20:07:19,因工作地点变动，转租现在的房间。明年4月份到期，可以直接跟房东签约。房间情况：今年3月份刚装...,2019,143147944,...,0,0,NaN,1880.0,1510.422269,1718.158282,1880.0,1,NaN,1880.0
1,【个人转租】租金1400，首月可补贴800，石厦地铁站，近福田CBD，随时可入住，有优惠，新...,寂静的风,11,2019-06-15 19:56,0,https://www.douban.com/group/topic/140787304/,2019-06-15 20:07:19,因为要和同事合租，只能转租，我还没住进去，随时可以入住。小区房，房间不大，公用厨房可以做饭，...,2019,140787304,...,0,0,NaN,1400.0,2296.383721,2176.810216,1600.0,1,NaN,1600.0
2,坪洲地铁站附近径贝新村一房一厅个人直租,豆友197023060,31,2019-06-15 19:46,0,https://www.douban.com/group/topic/141831059/,2019-06-15 20:07:19,径贝新村十二巷八号（导航至：勤业商务大厦）一房一厅出租，家电家私齐全，配备网络，监控，离坪洲...,2019,141831059,...,0,0,NaN,NaN,1672.030303,1718.158282,NaN,1,NaN,NaN
3,宝安灵芝地铁旁精装公寓转租,约翰君,26,2019-06-15 19:55,0,https://www.douban.com/group/topic/141853020/,2019-06-15 20:07:19,交通：灵芝地铁站旁，楼下诺铂广场公交站，去科技园方便，半个小时到科技园环境：新装修才半年，家...,2019,141853020,...,0,0,NaN,2450.0,1749.401961,1718.158282,NaN,1,NaN,2450.0
4,(转租)宝安坪洲地铁口两房一厅,MDZZᝰ,4,2019-06-15 19:46,0,https://www.douban.com/group/topic/143002741/,2019-06-15 20:07:19,转租两房一厅，阳光好，大阳台，离地铁非常近，宝安坪洲B出口，出门左拐就是地铁。现因舍友换工作...,2019,143002741,...,0,0,NaN,3700.0,1672.030303,1718.158282,NaN,2,1850.0,1850.0


In [86]:
gr = df_rent.loc[(df_rent.rent_type.isin(["转租","合租"]))&(df_rent.post_year=="2018")][["metro_station", "monthly_rent2"]].groupby("metro_station").agg({"metro_station":len,'monthly_rent2':pd.Series.median})
gr.index.name=""
gr.rename(columns={"metro_station": "post_cnt", "monthly_rent2": "monthly_rent"},inplace=True)
gr = gr.loc[(gr.index != "")]
gr.sort_values(by="post_cnt", ascending=False)

,post_cnt,monthly_rent
,,
固戍站,713,1500.0
西乡站,278,1600.0
坪洲站,235,1500.0
五和站,226,1400.0
大新站,214,2190.0
民治站,209,1680.0
西丽站,206,2000.0
南山站,202,2100.0
桃园站,187,1950.0


In [87]:
df_rent.loc[(df_rent.monthly_rent2 < 1000) & (df_rent.monthly_rent2 >= 500)].shape[0]

5620

In [88]:
df_rent.loc[(df_rent.monthly_rent2 < 1000) & (df_rent.monthly_rent2 >= 500)].head(10)

,title,author,response_count,response_time,page_num,url,dt,content,post_year,topic_id,...,girl_only,man_only,daily_rent,monthly_rent,ym_rent,ym_rent2,monthly_rent_modify,entire_room_num,entire_rent_split,monthly_rent2
9,个人转租！11号线桥头 麦当劳月租999全包 带家电,月亮那么圆,14,2019-06-15 19:58,0,https://www.douban.com/group/topic/143165249/,2019-06-15 20:07:19,地址在灶下路3巷三十号独栋公寓楼。楼下美食广场超市步行街水果店。什么都有很方便。走路到地铁站...,2019,143165249,...,0,0,NaN,999.0,1305.050000,1718.158282,NaN,1,NaN,999.0
13,布吉转租750!,远方,3,2019-06-15 19:51,0,https://www.douban.com/group/topic/143273145/,2019-06-15 20:07:19,三室两厅其中一个房间 有阳台 厨房 厕所 热水器 可以做饭 冰箱是我们自己买的 我的房间有空...,2019,143273145,...,0,0,NaN,750.0,1619.140496,1481.548148,750.0,1,NaN,750.0
73,转租创业花园单房950元双地铁深圳北红山,拌饭,None,2019-06-15 18:53,4,https://www.douban.com/group/topic/143317171/,2019-06-15 20:08:55,急需转租 电梯7楼 有阳光 空房 950房租 水6 电1.5 可以加我微信看视频 微信电话...,2019,143317171,...,0,0,NaN,950.0,1993.320388,1668.835795,950.0,1,NaN,950.0
115,【南山西丽】次卧合租800元/月,何必再,20,2019-06-15 18:44,4,https://www.douban.com/group/topic/142524802/,2019-06-15 20:08:55,小区为丽新小区，户型为三房两厅，这个房间是和我一起合住的。小区距离西丽地铁口仅3分钟左右，交...,2019,142524802,...,0,0,NaN,800.0,1989.780415,2192.551432,800.0,1,NaN,800.0
126,"【合租】主卧950元,坂田南坑村2室1厅露天大阳台,五和星河world附近",benarmy,52,2019-06-15 18:38,5,https://www.douban.com/group/topic/142753154/,2019-06-15 20:09:16,房子在7楼，2室1厅是房东那直接租的，光线很好。你住主卧，我住次卧，房间隔着露天大阳台，不怎...,2019,142753154,...,0,0,NaN,950.0,1452.718631,1481.548148,950.0,1,NaN,950.0
268,港隆城边上 三房侧卧招租，家具齐全，拎包入住 房租750,aliyunyun,9,2019-06-15 15:33,13,https://www.douban.com/group/topic/140936184/,2019-06-15 20:11:14,租金：750元房租；管理费，网费，水电平摊费用；主卧和一个侧卧各住一个小哥哥，房子有热水器，...,2019,140936184,...,0,0,NaN,750.0,1957.632277,1947.844182,NaN,3,250.0,750.0
283,港隆城边上三房侧卧 长租短租都可以,aliyunyun,11,2019-06-15 15:33,13,https://www.douban.com/group/topic/139387818/,2019-06-15 20:11:14,租金：750元房租；管理费，网费，水电平摊费用；主卧和一个侧卧各住一个小哥哥，房子有热水器，...,2019,139387818,...,0,0,NaN,750.0,1957.632277,1947.844182,NaN,3,250.0,750.0
399,木棉湾 南湾转租 短租6月底可入住,real__gappsong,4,2019-06-15 13:17,18,https://www.douban.com/group/topic/142862048/,2019-06-15 20:12:45,一房一厅 转的是房间 客厅还有一张床是我室友住 整个面积可观 大于40平方米 800出头转租...,2019,142862048,...,0,0,NaN,800.0,1240.888889,1481.548148,800.0,1,NaN,800.0
415,急转桥头地铁500米（同泰时代旁）单间一月800,小帆,8,2019-06-15 13:15,18,https://www.douban.com/group/topic/141607422/,2019-06-15 20:12:45,工作原因转租，房租签的3个月，5月20号刚签没多久，房租每月800，物业卫生100，要租的话...,2019,141607422,...,0,0,NaN,800.0,1305.050000,1718.158282,800.0,1,NaN,800.0
433,求租 求租 光明新区新地中央附近,小kuku,None,2019-06-15 12:57,19,https://www.douban.com/group/topic/143290526/,2019-06-15 20:13:06,工作调动 下个月要去新地中央上班 求租附近的房子 单间即可 700-1000左右的,2019,143290526,...,0,0,NaN,700.0,1957.632277,1069.230769,850.0,1,NaN,850.0


In [89]:
def get_monthly_rent_title(title):
    ntitle = parse_meaningless_content(title)
    s = re.search('[5-9]\d{2}|[1-9]\d{3,4}|\d{1,2}\.\d+[千k]|\d{1,2}[千k]|\d{1,2}\.\d+[万w]|\d{1,2}[万w]',ntitle)
    if s is None:
        return np.nan
    else:
        rent = parse_monthly_rent(s.group(0))
        return rent

In [90]:
df_rent_title = df_rent.loc[pd.isna(df_rent.monthly_rent2)].copy()
df_rent_title["monthly_rent2"] = df_rent_title["title"].apply(get_monthly_rent_title)

In [91]:
df_rent_title.loc[~pd.isna(df_rent_title.monthly_rent2)].head()

,title,author,response_count,response_time,page_num,url,dt,content,post_year,topic_id,...,girl_only,man_only,daily_rent,monthly_rent,ym_rent,ym_rent2,monthly_rent_modify,entire_room_num,entire_rent_split,monthly_rent2
150,坪洲合租单间 华侨新村4巷三栋（限女生）1900 招合拍小伙伴,Demi,1,2019-06-15 17:55,7,https://www.douban.com/group/topic/143078943/,2019-06-15 20:10:12,说明下：房东直招，房子接近120平，只会住两个人交通：1号线线上，到科技园高新园，8站地铁，...,2019,143078943,...,1,0,NaN,NaN,3487.965517,2192.551432,NaN,1,NaN,1900.0
229,【整租】个人原因需转租腾讯大厦附近开放式一室一厅一套·海景房全景落地窗·有合适的可联系看房哈...,豆友197747276,5,2019-06-15 16:36,11,https://www.douban.com/group/topic/143188074/,2019-06-15 20:10:50,我是租客，是个人转租房子的，因为工作变动需要转租腾讯大厦附近开放式一室一厅一套，带大阳台全景...,2019,143188074,...,0,0,NaN,NaN,1957.632277,1947.844182,NaN,1,NaN,1869.0
336,急租！！工作原因求短租一个月左右的房子，高新园地铁站附近.3000以内,RIP X,15,2019-06-15 14:44,15,https://www.douban.com/group/topic/143282530/,2019-06-15 20:12:08,本人急租，最好马上入住，短租1个月左右，个人女生，爱干净，就在科技园上班。救救孩子吧～,2019,143282530,...,0,0,NaN,NaN,3487.965517,2192.551432,NaN,1,NaN,3000.0
346,短租7月跟8月杨美地铁站两房一厅次卧，大阳台，电梯，热水器，洗衣机，冰箱。人均1100含水电...,Zack Handsom,3,2019-06-15 15:26,13,https://www.douban.com/group/topic/142961611/,2019-06-15 20:11:14,短租两个月，7月1号到8月31号杨美地铁站两室一厅的次卧带飘窗，大阳台，电梯，热水器，洗衣机...,2019,142961611,...,1,0,NaN,NaN,1780.562500,1481.548148,NaN,1,NaN,1100.0
682,【转租民乐花园两室一厅主卧￥1350】,南游的鱼儿,27,2019-06-15 08:18,27,https://www.douban.com/group/topic/140770823/,2019-06-15 20:16:22,1.房间有张1.2m的床(还有张1.5的床，想睡大床可以安装)、大衣柜、床头柜，书桌和椅子是...,2019,140770823,...,0,0,NaN,NaN,1541.690722,1668.835795,NaN,1,NaN,1350.0


In [92]:
df_rent = df_rent.merge(metro_line_agg, how="left", on="metro_station")

In [93]:
df_rent['year_month'] = df_rent['response_time'].apply(lambda x: x[0:7])

In [94]:
df_rent.rename(columns={"monthly_rent2": "room_rent"}, inplace=True)
df_rent_to_sql = df_rent[["topic_id", "title", "author", "content", "response_count", "response_time", "page_num", "url", "dt", "post_year","year_month",
                          "rent_type", "rent_style", "district", "metro_station", "line", "girl_only","man_only", "entire_room_num", "room_rent"]]

In [96]:
df_rent_pack = dict(df_rent_to_sql)

In [97]:
len(df_rent_pack['topic_id'])

125709

In [98]:
df_rent_pack.keys()

dict_keys(['topic_id', 'title', 'author', 'content', 'response_count', 'response_time', 'page_num', 'url', 'dt', 'post_year', 'year_month', 'rent_type', 'rent_style', 'district', 'metro_station', 'line', 'girl_only', 'man_only', 'entire_room_num', 'room_rent'])

In [127]:
sz_rent_group_v2 = db.douban_sz_rent_group_v2

In [143]:
sz_rent_group_v2 = db.douban_sz_rent_group_v2
key_list = list(df_rent_pack.keys())
for i in range(len(df_rent_pack['topic_id'])):
    item = {}
    for key in key_list:
        val = df_rent_pack[key][i]
        if isinstance(val, (np.float32, np.float64, np.int32, np.int64)):
            item[key] = '' if np.isnan(val) else  str(val)
        else:
            item[key] = val

    sz_rent_group_v2.insert_one(item)

In [ ]:
df_rent_to_sql.head()